In [0]:
%sql
use catalog `corerep`;
use schema `raw_data`;


/* Create Table Command 
* Get fixed income info */
create table if not exists raw_fixed_income
using delta
as
select * from `deutsche_boerse_market_data_and_services_eurex_open_interest_insights_fixed_income_free_sample`.`mds_platinum_sharing`.`eurex_oi_v4_fi`;

In [0]:
%sql
select *from corerep.raw_data.raw_fixed_income;

In [0]:
import re
# Read the CSV file into a DataFrame
df = spark.read.csv(
    "dbfs:/Volumes/databricks_bank_loan_modelling_dataset/v01/banking/loan-clean.csv",
    header=True,
    sep=",",
    inferSchema=True
)

# Minimal fix: rename columns to remove spaces and special characters
new_columns = [re.sub(r'[^a-zA-Z0-9_]', '_', col) for col in df.columns]
df = df.toDF(*new_columns)

# Write the DataFrame to the Unity Catalog table, overwriting if it exists
df.write.mode("overwrite").saveAsTable("corerep.raw_data.raw_loan")

In [0]:
%sql
select * from corerep.raw_data.raw_loan;

In [0]:
%sql
use catalog `corerep`;
use schema `raw_data`;
select current_catalog(), current_schema();
SHOW TABLES;

In [0]:
%sql
/* Describe Command */
DESCRIBE HISTORY corerep.raw_data.raw_loan;


In [0]:
%sql
CREATE TABLE IF NOT EXISTS corerep.raw_data.raw_country_risk
USING DELTA
AS
SELECT * FROM read_files('/Volumes/corerep/raw_data/country_data',
  format => 'csv',
  header => true,
  inferSchema => true
);

In [0]:
%sql
select * from corerep.raw_data.raw_country_risk;

In [0]:
%sql
describe detail corerep.raw_data.raw_loan;

In [0]:
dbutils.fs.ls("/Volumes/corerep/raw_data/country_data/db_create_raw_tbls.csv")

In [0]:
%fs ls "dbfs:/Volumes/corerep/raw_data/country_data/db_create_raw_tbls.csv"

In [0]:
%sql
/* Create raw tables for general ledger */

CREATE TABLE IF NOT EXISTS corerep.raw_data.raw_gl_balances
USING DELTA
AS
SELECT * FROM dataplatr_corp_oracle_financial_general_ledger_gl_analytics.marketplace.accounting_gl_balances;

In [0]:
%sql
select * from  corerep.raw_data.raw_gl_balances;

In [0]:
%sql
describe table corerep.raw_data.raw_gl_balances

In [0]:
%sql
SELECT SUM(cnt)
FROM (
  SELECT COUNT(*) AS cnt
  FROM corerep.raw_data.raw_gl_balances
  GROUP BY GLBalanceKey
  HAVING COUNT(*) = 1
) sub;